<a href="https://colab.research.google.com/github/2303A52060/onlinee_cource_2_year/blob/main/23_9_24_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
import pickle
def load_data(filepath):
    columns = ['target', 'id', 'date', 'flag', 'user', 'text']
    data = pd.read_csv(filepath, encoding = "latin-1", names=columns)
    return data
df = load_data("training.1600000.processed.noemoticon.csv")
df


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
42368,0,1676064993,Fri May 01 21:00:32 PDT 2009,NO_QUERY,lovehaightpeace,I wish Eric Draven was real.
42369,0,1676065032,Fri May 01 21:00:32 PDT 2009,NO_QUERY,bhillss,exam time
42370,0,1676065247,Fri May 01 21:00:34 PDT 2009,NO_QUERY,OxCheliJoxO,is gettin sick! Damn! Go figure I have big wee...
42371,0,1676065305,Fri May 01 21:00:35 PDT 2009,NO_QUERY,exeromai,Had to leave after shooting 1st 3 songs of Dea...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42373 entries, 0 to 42372
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  42373 non-null  int64 
 1   id      42373 non-null  int64 
 2   date    42373 non-null  object
 3   flag    42373 non-null  object
 4   user    42373 non-null  object
 5   text    42372 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [6]:
df['text'][5]

'@Kwesidei not the whole crew '

In [7]:
df['text'][7]

"@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?"

In [8]:
df['tweet'] = df['text'].str.lower()
df['tweet'][7]

"@loltrish hey  long time no see! yes.. rains a bit ,only a bit  lol , i'm fine thanks , how's you ?"

In [15]:
# Import the regular expression module
import re

# Convert all values in the 'tweet' column to strings
df['tweet'] = df['tweet'].astype(str)

# Apply the regular expression substitution to remove non-alphanumeric characters
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', "", x))

df['tweet'][5]
df['tweet'][7]

'loltrish hey  long time no see yes rains a bit only a bit  lol  im fine thanks  hows you '

In [16]:
df["tweet_tokens"] = df['tweet'].apply(lambda x:x.split())
df["tweet_tokens"][7]

['loltrish',
 'hey',
 'long',
 'time',
 'no',
 'see',
 'yes',
 'rains',
 'a',
 'bit',
 'only',
 'a',
 'bit',
 'lol',
 'im',
 'fine',
 'thanks',
 'hows',
 'you']

In [18]:
import nltk
nltk.download('stopwords')

lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [19]:
df['tweet_refine'] = df['tweet_tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['tweet_refine']

,tweet_refine
0,"[switchfoot, httptwitpiccomyzl, awww, thats, b..."
1,"[upset, cant, update, facebook, texting, might..."
2,"[kenichan, dived, many, times, ball, managed, ..."
3,"[whole, body, feels, itchy, like, fire]"
4,"[nationwideclass, behaving, im, mad, cant, see]"
...,...
42368,"[wish, eric, draven, real]"
42369,"[exam, time]"
42370,"[gettin, sick, damn, go, figure, big, weekend,..."
42371,"[leave, shooting, st, songs, death, cab, ben, ..."


In [22]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet') # Download the wordnet resource

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
len(stop_words)

df['tweet_refine'] = df['tweet_refine'].apply(lambda x: [lemma.lemmatize(word) for word in x])
stem = PorterStemmer()
df['tweet_refine'] = df['tweet_refine'].apply(lambda x: [stem.stem(word) for word in x])
df['tweet_refine'][5634]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


['cyclesoci',
 'comment',
 'neg',
 'stori',
 'httptinyurlcomcgqajm',
 'uk',
 'societi',
 'seemingli',
 'doesnt',
 'want',
 'slow']

In [21]:
X = df["tweet_refine"]
y = df['target']
len(X[2452])

12

In [23]:
tokenizer = Tokenizer(num_words=100000,oov_token="<OOV>")
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
len(X_tokenized[454])


8

In [24]:
X[2452]

['ahhh',
 'drafthouse',
 'surprise',
 'world',
 'premiere',
 'screening',
 'new',
 'star',
 'trek',
 'last',
 'night',
 'torchys']

In [25]:
X_padded = pad_sequences(X_tokenized, maxlen=50)
len(X_padded[534])

50

In [26]:
(X_padded)

array([[    0,     0,     0, ...,  5809,  1612,     5],
       [    0,     0,     0, ...,    10,   185,   799],
       [    0,     0,     0, ...,   358,     4,  7070],
       ...,
       [    0,     0,     0, ...,    43,  2629,  1090],
       [    0,     0,     0, ...,    11, 46955,  2945],
       [    0,     0,     0, ...,     0,     0,  8013]], dtype=int32)

In [27]:
y.unique()

array([0])

In [28]:
# Build LSTM model
def build_lstm_model(vocab_size, embedding_dim=100, max_len=50):
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_len),
        LSTM(128, return_sequences=True),
        LSTM(64),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
def train_model(model, X_train, y_train, X_val, y_val, epochs=5, batch_size=64):
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1
    )
    return history
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

46956

In [29]:
model = build_lstm_model(vocab_size)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.save('sentiment_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)